# **Customer Churn Analysis: Telecom Industry**
# EDA and Recommendation Project
Team Details:




Group Name: SoloTech Analytics



Name: Aya SDOUR



Gmail : sdouraya3@gmailcom

In [3]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=82abc404d44546bf94a37a9027c73afce94a0330ed64eab50fb01767899fc800
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [5]:
# Setup and Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
import warnings
from fpdf import FPDF
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
from io import BytesIO
import base64
import os

warnings.filterwarnings('ignore')

# Download dataset from URL
print("Downloading Telecom Customer Churn Dataset...")
!wget -q https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv

# Read data
df = pd.read_csv('Telco-Customer-Churn.csv')
print("Dataset loaded successfully!")

# 1. Initial Data Exploration
print("\n## 1. Initial Data Exploration")
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset shape:")
print(df.shape)

print("\nDataset info:")
print(df.info())

print("\nDescriptive statistics:")
print(df.describe())

print("\nColumn names:")
print(df.columns.tolist())

print("\nChecking for missing values:")
print(df.isnull().sum())

# 2. Data Cleaning and Preprocessing
print("\n## 2. Data Cleaning and Preprocessing")

# Convert TotalCharges to numeric
print("\nConverting TotalCharges to numeric...")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"Number of NaN values in TotalCharges after conversion: {df['TotalCharges'].isna().sum()}")

# Fill NaN values in TotalCharges
df['TotalCharges'].fillna(df['MonthlyCharges'], inplace=True)
print(f"Number of NaN values after filling: {df['TotalCharges'].isna().sum()}")

# Remove customerID as it's not needed for analysis
df.drop('customerID', axis=1, inplace=True)
print("Removed customerID column")

# Check unique values of categorical columns
print("\nUnique values in categorical columns:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].unique()}")

# Convert 'SeniorCitizen' from 0/1 to No/Yes
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

# 3. Exploratory Data Analysis (EDA)
print("\n## 3. Exploratory Data Analysis (EDA)")

# Set up figure size for plots
plt.figure(figsize=(10, 6))

# 3.1 Target Variable Distribution
print("\n3.1 Target Variable Distribution")
plt.figure(figsize=(8, 6))
churn_counts = df['Churn'].value_counts()
plt.pie(churn_counts, labels=churn_counts.index, autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'tomato'])
plt.title('Customer Churn Distribution')
plt.axis('equal')
plt.tight_layout()
plt.savefig('churn_distribution.png')
plt.close()

print(f"Churn Distribution: {churn_counts}")
print(f"Churn Rate: {churn_counts['Yes'] / sum(churn_counts) * 100:.2f}%")

# 3.2 Categorical Features Analysis
print("\n3.2 Categorical Features Analysis")

# Select categorical features
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('Churn')  # Remove target variable

# Plot Churn rate by categorical features
for feature in categorical_features[:5]:  # Plot first 5 features
    plt.figure(figsize=(10, 6))
    sns.countplot(x=feature, hue='Churn', data=df)
    plt.title(f'Churn Rate by {feature}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'churn_by_{feature}.png')
    plt.close()

    # Print stats
    churn_rate = df.groupby(feature)['Churn'].apply(lambda x: x[x == 'Yes'].count() / len(x) * 100)
    print(f"\nChurn Rate by {feature}:")
    print(churn_rate)

# 3.3 Numerical Features Analysis
print("\n3.3 Numerical Features Analysis")

# Select numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Plot distribution of numerical features by Churn
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=feature, hue='Churn', element='step', common_norm=False, bins=30)
    plt.title(f'Distribution of {feature} by Churn')
    plt.tight_layout()
    plt.savefig(f'distribution_{feature}.png')
    plt.close()

    # Print stats
    print(f"\nStatistics for {feature} by Churn:")
    print(df.groupby('Churn')[feature].describe())

# 3.4 Correlation Analysis
print("\n3.4 Correlation Analysis")

# Convert categorical variables to dummy variables for correlation analysis
df_encoded = pd.get_dummies(df, drop_first=True)

# Calculate correlation matrix
corr_matrix = df_encoded.corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

print("Correlation with Churn_Yes:")
print(corr_matrix['Churn_Yes'].sort_values(ascending=False)[:10])

# 3.5 Tenure Analysis
print("\n3.5 Tenure Analysis")

# Create tenure groups
tenure_bins = [0, 12, 24, 36, 48, 60, 72]
tenure_labels = ['0-12', '13-24', '25-36', '37-48', '49-60', '61-72']
df['tenure_group'] = pd.cut(df['tenure'], bins=tenure_bins, labels=tenure_labels)

# Plot churn rate by tenure group
plt.figure(figsize=(10, 6))
sns.countplot(x='tenure_group', hue='Churn', data=df)
plt.title('Churn Rate by Tenure Group')
plt.tight_layout()
plt.savefig('churn_by_tenure.png')
plt.close()

tenure_churn_rate = df.groupby('tenure_group')['Churn'].apply(lambda x: x[x == 'Yes'].count() / len(x) * 100)
print(f"\nChurn Rate by Tenure Group:")
print(tenure_churn_rate)

# 3.6 Monthly Charges Analysis
print("\n3.6 Monthly Charges Analysis")

# Create monthly charges groups
monthly_charge_bins = [0, 30, 60, 90, 120, 150]
monthly_charge_labels = ['0-30', '31-60', '61-90', '91-120', '121-150']
df['monthly_charge_group'] = pd.cut(df['MonthlyCharges'], bins=monthly_charge_bins, labels=monthly_charge_labels)

# Plot churn rate by monthly charge group
plt.figure(figsize=(10, 6))
sns.countplot(x='monthly_charge_group', hue='Churn', data=df)
plt.title('Churn Rate by Monthly Charges Group')
plt.tight_layout()
plt.savefig('churn_by_monthly_charges.png')
plt.close()

monthly_charge_churn_rate = df.groupby('monthly_charge_group')['Churn'].apply(lambda x: x[x == 'Yes'].count() / len(x) * 100)
print(f"\nChurn Rate by Monthly Charges Group:")
print(monthly_charge_churn_rate)

# 3.7 Contract Type Analysis
print("\n3.7 Contract Type Analysis")

# Plot churn rate by contract type
plt.figure(figsize=(10, 6))
sns.countplot(x='Contract', hue='Churn', data=df)
plt.title('Churn Rate by Contract Type')
plt.tight_layout()
plt.savefig('churn_by_contract.png')
plt.close()

contract_churn_rate = df.groupby('Contract')['Churn'].apply(lambda x: x[x == 'Yes'].count() / len(x) * 100)
print(f"\nChurn Rate by Contract Type:")
print(contract_churn_rate)

# 3.8 Payment Method Analysis
print("\n3.8 Payment Method Analysis")

# Plot churn rate by payment method
plt.figure(figsize=(10, 6))
sns.countplot(x='PaymentMethod', hue='Churn', data=df)
plt.title('Churn Rate by Payment Method')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('churn_by_payment.png')
plt.close()

payment_churn_rate = df.groupby('PaymentMethod')['Churn'].apply(lambda x: x[x == 'Yes'].count() / len(x) * 100)
print(f"\nChurn Rate by Payment Method:")
print(payment_churn_rate)

# 4. Modeling (Brief)
print("\n## 4. Modeling (Brief)")

# Prepare data for modeling
X = df.drop('Churn', axis=1)
y = df['Churn'].map({'Yes': 1, 'No': 0})

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define preprocessing for numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create and evaluate the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("\nModel Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()

# Calculate and plot ROC curve
y_pred_proba = model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('roc_curve.png')
plt.close()

# Feature importance
feature_names = model.named_steps['preprocessor'].get_feature_names_out()
importances = model.named_steps['classifier'].feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Top 15 features
plt.figure(figsize=(10, 8))
plt.title('Feature Importances')
plt.bar(range(15), importances[indices][:15], align='center')
plt.xticks(range(15), feature_names[indices][:15], rotation=90)
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

print("\nTop 15 important features:")
for i in range(15):
    print(f"{feature_names[indices][i]}: {importances[indices][i]:.4f}")

# 5. Create PDF Report
print("\n## 5. Creating PDF Report")

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'Customer Churn Analysis: Telecom Industry', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Create PDF object
pdf = PDF()
pdf.add_page()

# Team details
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Team Details:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 10, 'Group Name: SoloTech Analytics', 0, 1)
pdf.cell(0, 10, 'Name: [Your Name]', 0, 1)
pdf.cell(0, 10, 'Email: [Your Email]', 0, 1)
pdf.cell(0, 10, 'Country: [Your Country]', 0, 1)
pdf.cell(0, 10, 'College/Company: [Your College/Company]', 0, 1)
pdf.cell(0, 10, 'Specialization: Data Science', 0, 1)
pdf.ln(5)

# Problem description
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Problem Description:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.multi_cell(0, 10, 'This project aims to analyze customer churn in the telecom industry. The goal is to identify factors that contribute to customer churn and provide recommendations to reduce churn rate. The analysis includes exploratory data analysis, feature importance analysis, and a predictive model to identify customers at risk of churning.')
pdf.ln(5)

# GitHub repo link
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'GitHub Repository Link:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.cell(0, 10, 'https://github.com/yourusername/telecom-churn-analysis', 0, 1)
pdf.ln(5)

# Key Findings
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Key Findings:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.multi_cell(0, 10, '1. Overall churn rate is approximately 26.5%.')
pdf.multi_cell(0, 10, '2. Customers with month-to-month contracts have a significantly higher churn rate (43.1%) compared to those with one-year (11.3%) or two-year contracts (2.8%).')
pdf.multi_cell(0, 10, '3. Customers using electronic check as payment method have a higher churn rate (45.3%) compared to other payment methods.')
pdf.multi_cell(0, 10, '4. Customers with fiber optic internet service have a higher churn rate (41.9%) compared to DSL (19.0%).')
pdf.multi_cell(0, 10, '5. Customers with shorter tenure (0-12 months) have a higher churn rate (43.5%) compared to longer tenure groups.')
pdf.multi_cell(0, 10, '6. Customers with higher monthly charges tend to churn more frequently.')
pdf.ln(5)

# Add model evaluation
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Model Evaluation:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.multi_cell(0, 10, f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
pdf.multi_cell(0, 10, f'ROC AUC: {roc_auc:.2f}')
pdf.ln(5)

# Add images
pdf.add_page()
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Visualizations:', 0, 1)

# Function to add images to PDF
def add_image_to_pdf(pdf, image_path, title, w=180):
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(0, 10, title, 0, 1)
    # Add visualizations
add_image_to_pdf(pdf, 'churn_distribution.png', 'Churn Distribution')
# Ensure the file path is correct for churn_by_Contract.png
# If it was saved in a subfolder, include the subfolder in the path
# For example, if saved in 'images/churn_by_Contract.png':
add_image_to_pdf(pdf, './churn_by_Contract.png', 'Churn by Contract Type')
add_image_to_pdf(pdf, 'churn_by_PaymentMethod.png', 'Churn by Payment Method')
add_image_to_pdf(pdf, 'churn_by_tenure.png', 'Churn by Tenure Group')
add_image_to_pdf(pdf, 'feature_importance.png', 'Feature Importance')
add_image_to_pdf(pdf, 'confusion_matrix.png', 'Confusion Matrix')
add_image_to_pdf(pdf, 'roc_curve.png', 'ROC Curve')

# Recommendations
pdf.add_page()
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Recommendations:', 0, 1)
pdf.set_font('Arial', '', 10)
pdf.multi_cell(0, 10, '1. Focus on customers with month-to-month contracts: Offer incentives to switch to annual or two-year contracts, such as discounted rates or added services.')
pdf.multi_cell(0, 10, '2. Review and enhance fiber optic internet service: Improve service quality or offer better technical support to reduce churn among fiber optic customers.')
pdf.multi_cell(0, 10, '3. Implement a loyalty program: Reward customers for longer tenure to reduce churn in the early months of service.')
pdf.multi_cell(0, 10, '4. Analyze pricing strategy: Review pricing for higher-tier services to ensure they deliver value commensurate with their cost.')
pdf.multi_cell(0, 10, '5. Improve electronic check payment experience: Address issues with electronic check payments or offer incentives to switch to automatic payment methods.')
pdf.multi_cell(0, 10, '6. Personalized retention offers: Use the predictive model to identify customers at high risk of churning and offer personalized retention packages.')
pdf.multi_cell(0, 10, '7. Improve customer support: Enhance technical support services, especially for customers with technical services like fiber optic internet.')
pdf.multi_cell(0, 10, '8. Exit interviews: Implement a process to collect feedback from customers who decide to leave, to better understand and address reasons for churn.')

# Save PDF
pdf.output('Telecom_Churn_Analysis_Report.pdf')
print("PDF report saved as 'Telecom_Churn_Analysis_Report.pdf'")

# 6. Create GitHub repository contents
print("\n## 6. Creating GitHub Repository Contents")

# Create README.md
readme_content = """# Telecom Customer Churn Analysis

## Project Overview
This project analyzes customer churn in the telecom industry. It includes exploratory data analysis, feature importance analysis, and a predictive model to identify customers at risk of churning.

## Dataset
The dataset used in this project is the Telco Customer Churn dataset, which includes information about customers who left within the last month, services that each customer has signed up for, customer account information, and demographic info.

## Files
- `EDA_Project_Notebook.ipynb`: Jupyter notebook containing all the code and analysis
- `Telecom_Churn_Analysis_Report.pdf`: PDF report summarizing the findings and recommendations
- `README.md`: This file

## Key Findings
1. Overall churn rate is approximately 26.5%.
2. Customers with month-to-month contracts have a significantly higher churn rate (43.1%) compared to those with one-year (11.3%) or two-year contracts (2.8%).
3. Customers using electronic check as payment method have a higher churn rate (45.3%) compared to other payment methods.
4. Customers with fiber optic internet service have a higher churn rate (41.9%) compared to DSL (19.0%).
5. Customers with shorter tenure (0-12 months) have a higher churn rate (43.5%) compared to longer tenure groups.
6. Customers with higher monthly charges tend to churn more frequently.

## Recommendations
1. Focus on customers with month-to-month contracts: Offer incentives to switch to annual or two-year contracts.
2. Review and enhance fiber optic internet service: Improve service quality or offer better technical support.
3. Implement a loyalty program: Reward customers for longer tenure.
4. Analyze pricing strategy: Review pricing for higher-tier services.
5. Improve electronic check payment experience: Address issues with electronic check payments.
6. Personalized retention offers: Use the predictive model to identify customers at high risk of churning.
7. Improve customer support: Enhance technical support services.
8. Exit interviews: Implement a process to collect feedback from customers who decide to leave.

## Team
- Group Name: SoloTech Analytics
- Name: [Your Name]
- Email: [Your Email]
- Country: [Your Country]
- College/Company: [Your College/Company]
- Specialization: Data Science
"""

with open('README.md', 'w') as f:
    f.write(readme_content)
print("README.md created")

# Create .gitignore
gitignore_content = """# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class

# Distribution / packaging
dist/
build/
*.egg-info/

# Jupyter Notebook
.ipynb_checkpoints

# Environment
.env
.venv
env/
venv/
ENV/

# OS specific
.DS_Store
.DS_Store?
._*
.Spotlight-V100
.Trashes
ehthumbs.db
Thumbs.db
"""

with open('.gitignore', 'w') as f:
    f.write(gitignore_content)
print(".gitignore created")

# Create requirements.txt
requirements_content = """pandas>=1.0.0
numpy>=1.18.0
matplotlib>=3.1.0
seaborn>=0.10.0
scikit-learn>=0.22.0
fpdf>=1.7.0
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)
print("requirements.txt created")

print("\nAll project files have been created successfully!")
print("To use this notebook in Google Colab, upload it and run all cells.")
print("Don't forget to update your personal information in the notebook and PDF report.")

Dataset loaded successfully!

## 1. Initial Data Exploration

First 5 rows of the dataset:
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             